In [1]:
! git clone https://github.com/dnk3-skk4/Soccer-ball-detection-and-tracking.git

Cloning into 'Soccer-ball-detection-and-tracking'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 66 (delta 25), reused 4 (delta 1), pack-reused 0
Unpacking objects: 100% (66/66), done.


In [2]:
%cd Soccer-ball-detection-and-tracking

/content/Soccer-ball-detection-and-tracking


In [3]:
%cd data

/content/Soccer-ball-detection-and-tracking/data


In [4]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2022-11-06 16:25:24--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  17.4MB/s    in 15s     

2022-11-06 16:25:40 (15.9 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [5]:
import cv2
y3_weights_path = '/content/Soccer-ball-detection-and-tracking/data/yolov3.weights'
y3_config_path = '/content/Soccer-ball-detection-and-tracking/data/yolov3.cfg'
coco_classes_path = '/content/Soccer-ball-detection-and-tracking/data/coco.names' # file with multiple classes
classes = None
with open(coco_classes_path, 'rt') as f:
  classes = f.read().strip('\n').split('\n')

# load the network
net = cv2.dnn.readNetFromDarknet(y3_config_path, y3_weights_path)

In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from copy import deepcopy

In [7]:
# make sure the current directory is also where the folder the notebook resides in!
import os
os.getcwd()

'/content/Soccer-ball-detection-and-tracking/data'

In [8]:
def Set_Tracker(t_type_lst: list, t_type_index : int):
  # get the tracker list and get the current index
  tracker_name = t_type_lst[t_type_index]
  # Tracker creation
  tracker = None
  if tracker_name == t_type_lst[0]: 
    # BOOSTING
    tracker = cv2.TrackerBoosting_create()
  elif tracker_name == t_type_lst[1]:
    # MIL
    tracker = cv2.TrackerMIL_create()
  elif tracker_name == t_type_lst[2]:
    # KCF
    tracker = cv2.TrackerKCF_create()
  elif tracker_name == t_type_lst[3]:
    # TLD
    tracker = cv2.TrackerTLD_create()
  elif tracker_name == t_type_lst[4]:
    # MEDIANFLOW
    tracker = cv2.TrackerMedianFlow_create()
  elif tracker_name == t_type_lst[5]:
    # GOTURN
    tracker = cv2.TrackerGOTURN_create()
  elif tracker_name == t_type_lst[6]:
    # CSRT
    tracker = cv2.TrackerCSRT_create()
  elif tracker_name == t_type_lst[7]:
    tracker = cv2.TrackerMOSSE_create()
  else:
    print('Something went wrong initializing the tracker')
  return tracker
def Track_Object(dframe : np.array, tracker : 'tracker')->np.array:
  global ball_tracker_bbox, thickness, ball_bbox_upscale, downscale_size

  tracked, ball_tracker_bbox = tracker.update(dframe)
  FLAG_TRACKER_OK = tracked
  if not tracked:
    ball_tracker_bbox =  (-1,-1,-1,-1)
  return dframe

In [9]:
# Get the names of the output layers
def getOutputsNames(net):
    # Get the names of all the layers in the network
    layersNames = net.getLayerNames()
    # Get the names of the output layers, i.e. the layers with unconnected outputs
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Draw the predicted bounding box
def drawPred(frame, classId, conf, left, top, width, height):
    global bbox_detector_color, thickness

    # Draw a bounding box.
    cv2.rectangle(frame, (left, top), (left+width, top+height), bbox_detector_color, thickness, cv2.LINE_AA)
    label = '%.2f' % conf
        
    # Get the label for the class name and its confidence
    if classes:
        assert(classId < len(classes))
        label = '%s:%s' % (classes[classId], label)
        
    #Display the label at the top of the bounding box
    # labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    # top = max(top, labelSize[1])
    # cv2.rectangle(frame, (left, top - round(1.5*labelSize[1])), (left + round(1.5*labelSize[0]), top + baseLine), (255, 255, 255), cv2.FILLED)
    # cv2.putText(frame, label, (left, top), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 1)
    return

# Remove the bounding boxes with low confidence using non-maxima suppression
def postprocess(frame, outs):
    global ball_detector_bbox, FLAG_DETECT_OK
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]

    classIds = []
    confidences = []
    boxes = []
    # Scan through all the bounding boxes output from the network and keep only the
    # ones with high confidence scores. Assign the box's class label as the class with the highest score.
    classIds = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            if detection[4] > objectnessThreshold :
                scores = detection[5:]
                classId = np.argmax(scores)
                confidence = scores[classId]
                if confidence > confThreshold and classId == 32:
                    # 'classId == 32' are sports ball objects
                    # only interested in detected sports ball objects
                    center_x = int(detection[0] * frameWidth)
                    center_y = int(detection[1] * frameHeight)
                    width = int(detection[2] * frameWidth)
                    height = int(detection[3] * frameHeight)
                    left = int(center_x - width / 2)
                    top = int(center_y - height / 2)
                    classIds.append(classId)
                    confidences.append(float(confidence))
                    boxes.append([left, top, width, height])

    # Perform non maximum suppression to eliminate redundant overlapping boxes with
    # lower confidences.
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
#     print('number of indices: ' + str(len(indices)))
    
    if len(indices) <= 0:
      ball_boxes_bb = (-1,-1,-1,-1)
      FLAG_DETECT_OK = False
      return
    # bounding boxes for detection
    for i in indices:
        i = i[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        
        # get the bounding box
        ball_detector_bbox = (left, top, width, height)
    FLAG_DETECT_OK = True
    return
    
def Detect_Object(frame: np.array):
  # Create a 4D blob from a frame.
  blob = cv2.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop=False)

  # Sets the input to the network
  net.setInput(blob)

  # Runs the forward pass to get output of the output layers
  outs = net.forward(getOutputsNames(net))
  #outs = net.forward()

  # Remove the bounding boxes with low confidence
  postprocess(frame, outs)
  return frame

In [10]:
def EventKeyR_iniReset():
  global ini_frame_copy, ini_frame
  ini_frame = np.copy(ini_frame_copy)
  ini_flag = False
  return
def EventKeyEnter_ini():
  global ini_flag
  ini_flag = True
  return

In [11]:
def EventTrkChangeTrackerType(*args):
  '''
  Changes t_type_index to the value from the trackbar 
  '''
  global t_type_index, FLAG_CHANGE_TRACKER
  t_type_index = args[0]
  FLAG_CHANGE_TRACKER = True
  return

In [12]:
def ValidateTracker(tracker_bb:tuple, detector_bb:tuple, tracker: 'Tracker object', frame) -> tuple:
  '''
  If the tracker's center point goes beyond the detector's rectangle
  set the tracker to the detector's bounding box
  '''
  global t_type_index, t_type_lst
  print('tracker: ' + str(tracker_bb))
  print('detector: ' + str(detector_bb))
  t0, t1, tw, th = tracker_bb
  d0, d1, dw, dh = detector_bb
  
  if sum([d0, d1, dw, dh]) <= 0:
    return tracker, tracker_bb
#   t0, t1, tw, th = tracker_bb
#   d0, d1, dw, dh = detector_bb

  # get tracker center
  x, y = t0+(tw//2), t1+(th//2)

  # get detector left, top, right, bottom coordinates
  x1, y1, x2, y2 = d0, d1, d0+dw, d1+dh
  valid = PointInsideRectange(x1,y1,x2,y2, x, y)
  row, col, dim = frame.shape
  d0, d1, dw, dh = int(max(0, d0)), int(max(0, d1)), int(max(0, dw)), int(max(0, dh))
  if not valid and (0 <= d0 and 0 <= dw and d0 + dw <= col and 0 <= d1 and 0 <= dh and d1 + dh <= row):
    '''
    if the tracker went outside the detector rectangle and detector_bb won't cause initialization issues
    '''
    print(d0 > 0, d1 > 0, d0+dw > 0, d1+dh > 0)
    tracker = Set_Tracker(t_type_lst, t_type_index)
    _ = tracker.init(frame_downscale, (d0, d1, dw, dh))
  elif not valid:
    '''
    if the tracker was invalid but the detector_bb will cause an issue
    simply invalidate the current tracker_bb and wait later
    '''
    tracker_bb = (-1,-1,-1,-1)
  return tracker, tracker_bb


def PointInsideRectange(x1, y1, x2, y2, x, y) -> bool:
  if x > x1 and x < x2 and y > y1 and y < y2:
    return True
  else:
    return False

In [ ]:
# insert the video path here for ball tracking
path_vid = '/content/Soccer-ball-detection-and-tracking/4th_trim.mp4'
cap = cv2.VideoCapture(path_vid)

'''
+++ TRACKER configuration
'''
win_name = 'Tracker'
ini_win_name = 'Select object bounding box'
t_type_lst = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'CSRT', 'MOSSE']
t_type_index = 0 # used to change tracker type
t_type_text = t_type_lst[t_type_index]
t_type_min, t_type_max = 0, len(t_type_lst)-1
txt_font_size, txt_thickness = 2, 3

k = 0 # used for key press
k_timer = 1 # wait time for keyboard press
frame_counter = 0 # used for resetting the frame

cv2.namedWindow(win_name, cv2.WINDOW_AUTOSIZE)

# Bounding box
ball_detector_bbox = (-1, -1, -1, -1) # bounding box of Detector
ball_tracker_bbox = (-1, -1, -1, -1) # bounding box of Tracker

bbox_tracker_color = (255, 125, 0)
bbox_detector_color = (125, 255, 0)
FLAG_CHANGE_TRACKER = False
FLAG_TRACKER_OK = False
FLAG_DETECT_OK = False # Flag to use if ball was detected
detection_timer = 10 # Detect every Nth frame
thickness = 2
FLAG_INIT = False
downscale_size = 0.75
tracker = None
no_detect = 0

'''
--- TRACKER configurations
'''

'''
+++ YOLO v3 detector variables
'''
# Initialize the parameters
objectnessThreshold = 0.5 # Objectness threshold
confThreshold = 0.5       # Confidence threshold
nmsThreshold = 0.4        # Non-maximum suppression threshold
inpWidth = 416            # Width of network's input image
inpHeight = 416           # Height of network's input image

y3_weights_path = '/content/Soccer-ball-detection-and-tracking/data/yolov3.weights'
y3_config_path = '/content/Soccer-ball-detection-and-tracking/data/yolov3.cfg'
coco_classes_path = '/content/Soccer-ball-detection-and-tracking/data/coco.names' # file with multiple classes
ball_classes_path = './data/target.names' # only sports ball
choice_path = coco_classes_path
classes = None
with open(choice_path, 'rt') as f:
  classes = f.read().strip('\n').split('\n')

# load the network
net = cv2.dnn.readNetFromDarknet(y3_config_path, y3_weights_path)
# net = cv2.dnn.DNN_TARGET_OPENCL(y3_config_path, y3_weights_path)

'''
--- YOLO v3 Configuration
'''

if not cap.isOpened():
  print('Could not open the video')
else:
  cv2.createTrackbar('Trackers: ', win_name, t_type_min, t_type_max, EventTrkChangeTrackerType)
  while cap.isOpened() and k != 27:

    if k == ord('r'):
      '''
      When the 'r' keyboard button is pressed, reset the video
      '''
      # reset frame to position 0
      frame_counter = 0
      cap.set(cv2.CAP_PROP_POS_FRAMES, frame_counter)

    # read the frame  
    frameIsRead, f = cap.read()

    if not frameIsRead:
      print('Video read unsuccessful')
      break

    # keep track of frame tick
    frame_counter += 1
    # Attempt to down scale for FPS Boost
    row, col, dim = f.shape
    row, col = int(row*downscale_size), int(col*downscale_size)
    dim = (col, row)
    frame_downscale = cv2.resize(f, dim, interpolation = cv2.INTER_AREA)
    if not FLAG_INIT:
      ball_tracker_bbox = cv2.selectROI(ini_win_name, frame_downscale)
      tracker = Set_Tracker(t_type_lst, t_type_index)
      FLAG_INIT = tracker.init(frame_downscale, ball_tracker_bbox)
      if FLAG_INIT:
        print('Tracker Initialized')
      else:
        print('Tracker not initialized')
      cv2.destroyWindow(ini_win_name)
    
    # perform detection every Nth frames
    ball_detector_bbox = (-1, -1, -1, -1)
    if (frame_counter - 1) % detection_timer == 0:
      # ball_detector_bbox gets updated inside Detect_Object
      detect_frame = Detect_Object(frame_downscale)
      if not FLAG_DETECT_OK:
        no_detect += 1
      else:
        no_detect = 0

    # ball_tracker_bbox gets updated inside Track_Objects
    track_frame = Track_Object(frame_downscale, tracker)

    tracker, ball_tracker_bbox = ValidateTracker(ball_tracker_bbox, ball_detector_bbox, tracker, frame_downscale)
    t0, t1, t2, t3 = ball_tracker_bbox
    d0, d1, d2, d3 = ball_detector_bbox

    print('Detector: ' + str(sum(ball_detector_bbox)))
    print('Tracker: ' + str(sum(ball_tracker_bbox)))
    print('\n')
    if sum(ball_detector_bbox) >= 0:
      cv2.rectangle(frame_downscale, (int(d0), int(d1)), (int(d0+d2), int(d1+d3)), bbox_detector_color, thickness, cv2.LINE_AA)
    if sum(ball_tracker_bbox) >= 0 and no_detect < 2:
      cv2.rectangle(frame_downscale, (int(t0), int(t1)), (int(t0+t2), int(t1+t3)), bbox_tracker_color, thickness, cv2.LINE_AA)
      cv2.putText(frame_downscale, 'Tracker: ' + str(tracker),(10,row), cv2.FONT_HERSHEY_PLAIN, txt_font_size, (255, 255, 255), txt_thickness, cv2.LINE_AA)
    else:
      cv2.putText(frame_downscale, 'No object tracked',(10,row), cv2.FONT_HERSHEY_PLAIN, txt_font_size, (255, 255, 255), txt_thickness, cv2.LINE_AA)
    cv2.imshow(win_name, frame_downscale)
    
    # read keyboard input
    k = cv2.waitKey(k_timer)

# release of the cap object
cap.release()
# close all windows
cv2.destroyAllWindows()